### notes

In [1]:
from BaseGraph import *
from Cells import *
from RNNs import *
from cswNetEngine import *
from analysis import *
from Trainer import *

%load_ext autoreload
%reload_ext autoreload
%autoreload 2

import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import colors
%matplotlib inline

/Users/abeukers/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## single block single graph

currently pred_arr concatenates blocks. eventually i'll want to have a separate dimension for block (i) to do some between block averaging and (ii) so that i can just pass the array with predictions  (pred_arr) into functions. keep in mind this will require changing the function that makes the dataframe with predictions (pred_df)

In [2]:
net_size = 10
cell = CustomLSTM(net_size,net_size,net_size,net_size)

In [5]:
rnn = RNN_fullstory(cell)

In [6]:
net_size = 10
saving_name="lstm%i+"%net_size

cell = CustomLSTM(net_size,net_size,net_size,net_size)
rnn = RNN_fullstory(cell)
trainer = Trainer(rnn)


In [13]:
X,Y = trainer.gen_train_data('location.latent.true')

In [15]:
Y.shape

(6, 2)

In [106]:
blocking = [(2,100),(3,100)]
eval_data,pred_data = trainer.sesstrain(blocking)

sess 0
0 0
0 10
0 20
0 30
0 40
0 50
0 60
0 70
0 80
0 90
1 0
1 10
1 20
1 30
1 40
1 50
1 60
1 70
1 80
1 90
sess 1
0 0
0 10
0 20
0 30
0 40
0 50
0 60
0 70
0 80
0 90
1 0
1 10
1 20
1 30
1 40
1 50
1 60
1 70
1 80
1 90
2 0
2 10
2 20
2 30
2 40
2 50
2 60
2 70
2 80
2 90


In [ ]:
# data has: (num sessions, num blocks, num evals)

In [107]:
eval_data[0]['loss'].shape

(2, 10)

In [15]:
pred_arr_L[0].shape,pred_arr_L[0].dtype

((20,), dtype([('loss', '<f4'), ('acc', '<f4', (2,)), ('cond', '<U25')]))

In [27]:
np.array(pred_arr_L)[1]['loss']

array([3.2358634 , 1.3414258 , 1.305071  , 1.2717901 , 1.2232074 ,
       1.1979767 , 1.1907247 , 1.2220477 , 1.2320894 , 1.2246588 ,
       1.0970117 , 1.228598  , 1.1590317 , 1.1482819 , 1.1082064 ,
       1.1266599 , 1.0244309 , 0.9744117 , 0.9572101 , 1.1576566 ,
       0.9461817 , 0.9309623 , 1.1381541 , 1.1647217 , 1.1140026 ,
       1.0729734 , 0.880825  , 0.9721715 , 1.0242225 , 1.0197194 ,
       3.9310162 , 1.2516822 , 1.1370596 , 1.1459615 , 1.1769618 ,
       1.096819  , 1.0727191 , 0.9996252 , 0.9659586 , 0.86335486,
       0.83780473, 0.78992826, 0.76253873, 1.0449356 , 0.8912707 ,
       0.9020014 , 1.0249184 , 0.95279163, 0.8220792 , 0.97476333,
       0.8848445 , 0.8205289 , 0.9246387 , 0.83274335, 0.74657196,
       0.6776693 , 0.73429567, 0.67735285, 0.6472662 , 0.80419165,
       3.395594  , 0.93178016, 0.753143  , 0.69345826, 0.7334761 ,
       0.718039  , 0.6047781 , 0.7214592 , 0.59450686, 0.6370217 ,
       0.65363127, 0.6792083 , 0.7235221 , 0.67136806, 0.73628

In [ ]:
net_size = 10
saving_name="lstm%i+"%net_size
cell = CustomLSTM(net_size,net_size,net_size)
rnn = RNN_fullstory(cell)

trainer = Trainer(rnn)
trainer.rnn.reinitialize()

num_blocks = 1000
num_epochs = 1 # per block
eval_arr,pred_arr = trainer.blocktrain(num_blocks,num_epochs)

## evaluation

In [ ]:
total_num_evals = num_blocks*num_evals_per_block
acc_plt = mov_avg(eval_arr['acc'],15)
xplt = np.arange(0,total_num_evals,len(acc_plt))

In [ ]:
plt_eval(eval_arr,num_blocks,num_epochs)

# exploring different training regimes

- what happens if i first train intrlv/block on A and B, then train only on B.
- does intrlv or block show different resistance to interference? 

In [73]:
csw.conditioned = True
csw.cond_dict

{'location.latent.false': 15, 'location.latent.true': 14}

In [42]:
# initialize network
net_size = 10
net_name="lstm%i"%net_size

cell = CustomLSTM(net_size,net_size,net_size)
rnn = RNN_fullstory(cell)
trainer = Trainer(rnn)
# look at evaluations on the following filler
eval_fillerid = trainer.csw.cond_dict['location.latent.false']

15

## blocked

In [ ]:
trainer.rnn.reinitialize()
block_blocking = [(20,200)]
# train
block_arr_L = trainer.sesstrain(block_blocking)

In [ ]:
# make array into dataframe
block_df = get_pred_df(block_arr_L,block_blocking)
# code network responses
coded_block_df = code_responses(block_df,trainer,eval_fillerid)

In [ ]:
plt_avg_predictions(coded_block_df,block_blocking)
plt.savefig('figures/'+net_name+'-2sch-'+'-blocked')

## interleave

In [ ]:
trainer.rnn.reinitialize()
intrlv_blocking = [(10000,1),(1,5000)]
# train
intrlv_arr_L = trainer.sesstrain(intrlv_blocking)

In [ ]:
# make array into dataframe
intrlv_df = get_pred_df(intrlv_arr_L,intrlv_blocking)
# code network responses
coded_intrlv_df = code_responses(intrlv_df,trainer,eval_fillerid)

In [ ]:
plt_avg_predictions(coded_intrlv_df,intrlv_blocking)
plt.savefig('figures/'+net_name+'-2sch-'+'-intrlv')